In [1]:
import pandas as pd
df = pd.read_csv('data/neos.csv')
df.head()

/Users/dmitrymikhaylov/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,5,16,23,24,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,spkid,full_name,pdes,name,prefix,neo,pha,H,G,...,n_obs_used,n_del_obs_used,n_dop_obs_used,condition_code,rms,two_body,A1,A2,A3,DT
0,a0000433,2000433,433 Eros (A898 PA),433,Eros,NaN,Y,N,10.4,0.46,...,8767,4.0,2.0,0.0,0.28397,NaN,NaN,NaN,NaN,NaN
1,a0000719,2000719,719 Albert (A911 TB),719,Albert,NaN,Y,N,15.5,NaN,...,1874,NaN,NaN,0.0,0.39148,NaN,NaN,NaN,NaN,NaN
2,a0000887,2000887,887 Alinda (A918 AA),887,Alinda,NaN,Y,N,13.8,-0.12,...,1424,NaN,NaN,0.0,0.48159,NaN,NaN,NaN,NaN,NaN
3,a0001036,2001036,1036 Ganymed (A924 UB),1036,Ganymed,NaN,Y,N,9.4,0.30,...,6249,0.0,1.0,0.0,0.38525,NaN,NaN,NaN,NaN,NaN
4,a0001221,2001221,1221 Amor (1932 EA1),1221,Amor,NaN,Y,N,17.7,NaN,...,540,NaN,NaN,0.0,0.45568,NaN,NaN,NaN,NaN,NaN


In [16]:
df.loc[df['pdes']=="2005 OE3"] 

,id,spkid,full_name,pdes,name,prefix,neo,pha,H,G,...,n_obs_used,n_del_obs_used,n_dop_obs_used,condition_code,rms,two_body,A1,A2,A3,DT
4523,bK05O03E,3285299,(2005 OE3),2005 OE3,NaN,NaN,Y,Y,20.3,NaN,...,263,3.0,2.0,0.0,0.42047,NaN,NaN,NaN,NaN,NaN


In [14]:
df2 = pd.DataFrame.from_dict(data = all_cads['data'])
df2.columns=all_cads['fields']
df2.loc[df2['des']=="2005 OE3"] 

,des,orbit_id,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h
1,2005 OE3,52,2415020.606013490,1900-Jan-01 02:33,0.414975519685102,0.414968315685577,0.414982724454678,17.918395877175,17.9180375373357,< 00:01,20.3
2703,2005 OE3,52,2415638.643480293,1901-Sep-11 03:27,0.34797461671528,0.347954745942431,0.347994488397595,14.0935437583352,14.0930004417598,00:02,20.3
3190,2005 OE3,52,2415751.735938334,1902-Jan-02 05:40,0.386675150770562,0.386671938415327,0.38667836409341,14.0377242753077,14.0372333934531,< 00:01,20.3
5793,2005 OE3,52,2416349.385796133,1903-Aug-22 21:16,0.117894766431221,0.117874009729329,0.117915524843601,15.0707465101125,15.0692468091358,00:02,20.3
6410,2005 OE3,52,2416490.064411213,1904-Jan-10 13:33,0.462065332694496,0.462055983946402,0.462074682114722,13.2336157390228,13.233179988817,00:02,20.3
8959,2005 OE3,52,2417067.069062581,1905-Aug-09 13:39,0.116480948712595,0.116460480733609,0.116501417017621,18.6542631613291,18.653036870068,00:02,20.3
12120,2005 OE3,52,2417785.732636105,1907-Jul-29 05:35,0.348725691079108,0.34870638697577,0.348744995188235,23.715547577644,23.7152253979191,00:01,20.3
85625,2005 OE3,52,2433650.449645088,1951-Jan-03 22:47,0.451554749366397,0.4515478592583,0.45156163970743,19.932056298779,19.9317602569611,< 00:01,20.3
88813,2005 OE3,52,2434287.785917512,1952-Oct-02 06:52,0.430703448629083,0.430692688699637,0.430714209033346,14.4174324849901,14.4170033908891,00:02,20.3
89249,2005 OE3,52,2434378.610133168,1953-Jan-01 02:39,0.378918850762934,0.378917747309762,0.378919954649343,15.1258294688387,15.1253645751829,< 00:01,20.3


In [387]:
#from helpers import cd_to_datetime, datetime_to_str

In [5]:
class NearEarthObject:
    """A near-Earth object (NEO).

    An NEO encapsulates semantic and physical parameters about the object, such
    as its primary designation (required, unique), IAU name (optional), diameter
    in kilometers (optional - sometimes unknown), and whether it's marked as
    potentially hazardous to Earth.

    A `NearEarthObject` also maintains a collection of its close approaches -
    initialized to an empty collection, but eventually populated in the
    `NEODatabase` constructor.
    """

    def __init__(self, designation, name=None, diameter=float('nan'), hazardous='N'):
        """Create a new `NearEarthObject`.
        
        :param info: 
        required unique designation (str), 
        optional name (str), diameter (float), hazardous (bool)
        """
        
        self.designation = str(designation)
        self.name = str(name).title()
        self.diameter = diameter
        self.hazardous = hazardous
        
        self.approaches = []

    @property
    def fullname(self):
        """Return a representation of the full name of this NEO."""
        
        if self.name is None:
            return self.designation      
        else:
            return self.designation + " (" + self.name + ")"
    
    @property
    def hazard(self):
        """Return 'is/is not' depending on boolean value of hazardous."""
        
        if self.hazardous == 'N':
            hazard = 'is not'
        else:
            hazard = 'is'
        return hazard

    def __str__(self):
        """Return `str(self)`."""
        formatted_string = "NEO {} has a diameter {} km and {} potentially hazardous.".format(
        self.fullname, self.diameter, self.hazard)

        return formatted_string



In [6]:

class CloseApproach:
    """A close approach to Earth by an NEO.

    A `CloseApproach` encapsulates information about the NEO's close approach to
    Earth, such as the date and time (in UTC) of closest approach, the nominal
    approach distance in astronomical units, and the relative approach velocity
    in kilometers per second.

    A `CloseApproach` also maintains a reference to its `NearEarthObject` -
    initally, this information (the NEO's primary designation) is saved in a
    private attribute, but the referenced NEO is eventually replaced in the
    `NEODatabase` constructor.
    """

    def __init__(self, neo, time, distance=float('nan'), velocity=float('nan')):
        """Create a new `CloseApproach`.

        :param info: 
        required unique designation (str), can be passed from the neo type'
        required date (str);
        optional: distance (float), velocity (bool), neo (neo).
        """

        self.neo = neo
        self._designation = neo.designation
        self.time = cd_to_datetime(time)
        
        self.distance = distance
        self.velocity = velocity

        
    @property
    def time_str(self):
        """Return a formatted representation of this `CloseApproach`'s approach time.

        The value in `self.time` should be a Python `datetime` object. While a
        `datetime` object has a string representation, the default representation
        includes seconds - significant figures that don't exist in our input
        data set.

        The `datetime_to_str` method converts a `datetime` object to a
        formatted string that can be used in human-readable representations and
        in serialization to CSV and JSON files.
        """

        return datetime_to_str(self.time)

    
    def __str__(self):
        """Return `str(self)`."""
        return (f"At {self.time_str} {self.neo.fullname}, "
                f"approaching with speed {self.velocity} and distance {self.distance}")

    def __repr__(self):
        """Return `repr(self)`, a computer-readable string representation of this object."""
        return (f"CloseApproach(time={self.time_str}, distance={self.distance:.2f}, "
                f"velocity={self.velocity:.2f}, neo={self.neo!r})")


### Loading datasets with `extract.py`

In [7]:
"""Extract data on near-Earth objects and close approaches from CSV and JSON files.

The `load_neos` function extracts NEO data from a CSV file, formatted as
described in the project instructions, into a collection of `NearEarthObject`s.

The `load_approaches` function extracts close approach data from a JSON file,
formatted as described in the project instructions, into a collection of
`CloseApproach` objects.

The main module calls these functions with the arguments provided at the command
line, and uses the resulting collections to build an `NEODatabase`.

You'll edit this file in Task 2.
"""
import csv
import json

#from models import NearEarthObject, CloseApproach


neo_csv_path = 'data/neos.csv'

def load_neos(neo_csv_path):
    """Read near-Earth object information from a CSV file.

    :param neo_csv_path: A path to a CSV file containing data about near-Earth objects.
    :return: A collection of `NearEarthObject`s.
    """
    # List below will store instanciated NEOs objects 
    # new_neo = NearEarthObject(designation, name=None, diameter=float('nan'), hazardous=False)
    neos = []
    with open(neo_csv_path, 'r') as file:
        reader = csv.DictReader(file)
        for line in reader:
            neos.append(NearEarthObject(designation=line['pdes'], 
                                        name=line['name'], 
                                        diameter=line['diameter'], 
                                        hazardous=['pha']
                                       )
                       )
    return neos

cad_json_path = 'data/cad.json'
def load_approaches(cad_json_path):
    """Read close approach data from a JSON file.

    :param neo_csv_path: A path to a JSON file containing data about close approaches.
    :return: A collection of `CloseApproach`es.
    """
    with open(cad_json_path, 'r') as file:
        cads = json.load(file)
    return cads

#(self, neo, time, distance=float('nan'), velocity=float('nan')

In [8]:
nn = load_neos(neo_csv_path)[:10]

In [9]:
for n in nn:
    print(n.designation)

433
719
887
1036
1221
1566
1580
1620
1627
1685


In [10]:
all_cads = load_approaches(cad_json_path)

In [11]:
all_cads['fields'] # select des, cd, dist, v_rel

['des',
 'orbit_id',
 'jd',
 'cd',
 'dist',
 'dist_min',
 'dist_max',
 'v_rel',
 'v_inf',
 't_sigma_f',
 'h']

In [12]:
all_cads['data'][0]

['170903',
 '105',
 '2415020.507669610',
 '1900-Jan-01 00:11',
 '0.0921795123769547',
 '0.0912006569517418',
 '0.0931589328621254',
 '16.7523040362574',
 '16.7505784933163',
 '01:00',
 '18.1']

### Encapsulate the data in a `NEODatabase`

In [ ]:
"""A database encapsulating collections of near-Earth objects and their close approaches.

A `NEODatabase` holds an interconnected data set of NEOs and close approaches.
It provides methods to fetch an NEO by primary designation or by name, as well
as a method to query the set of close approaches that match a collection of
user-specified criteria.

Under normal circumstances, the main module creates one NEODatabase from the
data on NEOs and close approaches extracted by `extract.load_neos` and
`extract.load_approaches`.

You'll edit this file in Tasks 2 and 3.
"""

class NEODatabase:
    """A database of near-Earth objects and their close approaches.

    A `NEODatabase` contains a collection of NEOs and a collection of close
    approaches. It additionally maintains a few auxiliary data structures to
    help fetch NEOs by primary designation or by name and to help speed up
    querying for close approaches that match criteria.
    """
    def __init__(self, neos, approaches):
        """Create a new `NEODatabase`.

        As a precondition, this constructor assumes that the collections of NEOs
        and close approaches haven't yet been linked - that is, the
        `.approaches` attribute of each `NearEarthObject` resolves to an empty
        collection, and the `.neo` attribute of each `CloseApproach` is None.

        However, each `CloseApproach` has an attribute (`._designation`) that
        matches the `.designation` attribute of the corresponding NEO. This
        constructor modifies the supplied NEOs and close approaches to link them
        together - after it's done, the `.approaches` attribute of each NEO has
        a collection of that NEO's close approaches, and the `.neo` attribute of
        each close approach references the appropriate NEO.

        :param neos: A collection of `NearEarthObject`s.
        :param approaches: A collection of `CloseApproach`es.
        """
        self._neos = neos
        self._approaches = approaches

        # TODO: What additional auxiliary data structures will be useful?

        # TODO: Link together the NEOs and their close approaches.

    def get_neo_by_designation(self, designation):
        """Find and return an NEO by its primary designation.

        If no match is found, return `None` instead.

        Each NEO in the data set has a unique primary designation, as a string.

        The matching is exact - check for spelling and capitalization if no
        match is found.

        :param designation: The primary designation of the NEO to search for.
        :return: The `NearEarthObject` with the desired primary designation, or `None`.
        """
        # TODO: Fetch an NEO by its primary designation.
        return None

    def get_neo_by_name(self, name):
        """Find and return an NEO by its name.

        If no match is found, return `None` instead.

        Not every NEO in the data set has a name. No NEOs are associated with
        the empty string nor with the `None` singleton.

        The matching is exact - check for spelling and capitalization if no
        match is found.

        :param name: The name, as a string, of the NEO to search for.
        :return: The `NearEarthObject` with the desired name, or `None`.
        """
        # TODO: Fetch an NEO by its name.
        return None

    def query(self, filters=()):
        """Query close approaches to generate those that match a collection of filters.

        This generates a stream of `CloseApproach` objects that match all of the
        provided filters.

        If no arguments are provided, generate all known close approaches.

        The `CloseApproach` objects are generated in internal order, which isn't
        guaranteed to be sorted meaninfully, although is often sorted by time.

        :param filters: A collection of filters capturing user-specified criteria.
        :return: A stream of matching `CloseApproach` objects.
        """
        # TODO: Generate `CloseApproach` objects that match all of the filters.
        for approach in self._approaches:
            yield approach


In [ ]:
with open("config.json") as f:
    data = json.load(f)
    for (k, v) in data.items():
        globals()[k] = v
        print(f"Loaded {k}:\t{v}")

In [565]:
cat_string = "--Whiskers--, --Spot--, --Meowmeow--, --Tiger--, --Kitty--, --Henry--, --Mr.Paws--"

In [568]:
cat_list = cat_string.split(',')

In [580]:
from random import choice
print(cat_list[1][3:-2])

Spot


In [594]:
i = choice(range(len(cat_list)))
print(cat_list[i][3:-2], "is a good cat")

Tiger is a good cat


In [605]:
(5*(40 * 10 * 10 * 10 * 30)) / (40 + 10 + 10 + 10 + 30)

60000.0

In [609]:
(30-12)/(6*12)

0.25

In [610]:
853 * 12

10236

### Example of class methods to dynamically create new classes:

In [611]:
class Dog():

    def __init__(self, name:str, age:int,
                 breed:str, weight:int):
        """Create a new dog"""
        self.breed = breed
        self.weight = weight
        self.name = name
        self.age = age
        
    def __gt__(self, other):
        return self.age > other.age
    
    def __str__(self):
        return str(self.name)
    
    def speak(self) -> None:
        """Make the dog bark"""
        print(f'{self.name} says, "woof"')
    @classmethod    
    def spawn(cls, name, mother, father):
        breed = mother.breed
        if mother.breed != father.breed:
            breed = f'{mother.breed}-{father.breed} Mix'
        weight = (mother.weight + father.weight) / (2 * 10)
        return cls(name, 0, breed, weight)

if __name__ == "__main__":
    sally = Dog('Sally', 6, 'chihuahua', 7)
    henry = Dog('Henry', 7, 'terrier', 15)
    trixy = Dog.spawn('Trixy', sally, henry)
    print(trixy.breed)

chihuahua-terrier Mix


In [612]:
trixy.weight

1.1

In [618]:
(9257+725) / 12

831.8333333333334

In [621]:
199 * 12


2388

In [622]:
18 + 16 + 5

39